In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['savefig.dpi'] = 144

In [2]:
import pandas as pd
import sklearn.datasets

# **Denetimli Makine Öğrenimi için Metrikler**

Denetimli makine öğreniminin çözmeye çalıştığı genel sorun, çeşitli değişkenlerin ölçümünü bir hedef değer veya sınıfla eşleştirmektir. Örneğin, konuşulan dili metne dönüştürmek, mahalle olanaklarına dayalı olarak ev değerlerini tahmin etmek veya hileli işlemleri tespit etmek için denetimli makine öğrenimini kullanabiliriz. Modelimizin başarılı olup olmadığını değerlendirmek için, verilen görev için başarının ne olduğunu resmi olarak tanımlamamız gerekir. Bu not defterinde, model performansı için birkaç yaygın ölçütü inceleyeceğiz.


Örnek olarak Kaliforniya konut veri setini kullanacağız. Kaliforniya konut veri seti, Kaliforniya'nın çeşitli nüfus sayım blokları için ortalama ev yaşı, ortalama oda sayısı, konum ve diğer niteliklerin ölçümlerini içerir.

In [3]:
!wget http://dataincubator-wqu.s3.amazonaws.com/caldata/cal_housing.pkz -nc -P ~/scikit_learn_data/


--2023-02-15 19:30:10--  http://dataincubator-wqu.s3.amazonaws.com/caldata/cal_housing.pkz
Resolving dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)... 52.217.135.185, 52.217.232.217, 52.216.205.195, ...
Connecting to dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)|52.217.135.185|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 366863 (358K) [binary/octet-stream]
Saving to: ‘/root/scikit_learn_data/cal_housing.pkz’

cal_housing.pkz     100%[===================>] 358.26K  --.-KB/s    in 0.1s    

2023-02-15 19:30:11 (2.79 MB/s) - ‘/root/scikit_learn_data/cal_housing.pkz’ saved [366863/366863]



In [4]:
from sklearn.datasets import fetch_california_housing

cali_data = fetch_california_housing()

print(cali_data.DESCR)

cali_df = pd.DataFrame(cali_data.data, columns=cali_data.feature_names)
cali_df.head()

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


Yukarıdaki veri çerçevesinde, her sütun bir özellik (yani bir değişken) ve her satır bir gözlemdir (yani bir ölçüm). Başka bir deyişle, medyan gelir ve ortalama oda sayısı gibi şeyler özelliklerdir, özelliklerin bir ölçümüne sahip olduğumuz her nüfus sayımı bloğu ise bir gözlemdir. Ayrıca, her mahalle için medyan ev değeri olan bir hedef etiket vektörümüz var. Toplamda 13 özelliğimiz ve etiketli 506 gözlemimiz var.


In [5]:
print(cali_data.data.shape)
print(cali_data.target.shape)

(20640, 8)
(20640,)


Gerçek sayı değerlerini (örneğin ev fiyatı) tahmin etmek için bir model tasarladığımızda, modelimiz bir regresyon modelidir. Alternatif olarak, "pahalı mahalle" ve "ucuz mahalle" gibi kategorik etiketleri tahmin etmek için bir model tasarlayabiliriz. Bu bir sınıflandırma modeli olacaktır. Denetimli makine öğrenimi görevlerinin çoğu regresyon veya sınıflandırma kategorisine girer.

Metriğimizi bir maliyet fonksiyonu tanımlamak için kullanıyoruz.(C diyelim). Gradyan inişini gerçekleştirmek için, C'nin türevini sayısal olarak değerlendiririz model parametrelerimize göre.

# **Regresyon Görevleri için Metrikler**

Doğrusal regresyonla ilgili not defterinde, trend çizgimizin nasıl performans gösterdiğine ilişkin bir ölçüt olarak ortalama karesel hatayı (MSE) tanıtmıştık. Bu bizi, model parametrelerimize bağlı olan skaler bir fonksiyon olan bir maliyet fonksiyonu tanımlamaya yöneltti. Gradyan inişi kullanarak maliyet fonksiyonunu minimize ettik. Hangi sorunu çözmeye çalıştığımıza ve neyi optimize etmek istediğimize bağlı olarak farklı ölçütler seçebiliriz.

Ortalama karesel hata (MSE) regresyon için en yaygın ölçütlerden biridir.

Model parametrelerimizi ayarlayarak MAE'yi minimize ettiğimizde, modelimiz aykırı değerlerden MSE'yi kullandığımızdan daha az etkilenecektir. Bunun nedeni, aykırı değerlerden kaynaklanan hata terimlerinin (genellikle büyük olacaktır) MAE'ye kare olarak değil doğrusal olarak girmesidir.

Bir metrik seçerken göz önünde bulundurulması gereken önemli bir husus, verilerin nasıl ölçeklendirildiğidir.

In [ ]:
from sklearn import metrics
import numpy as np

y = np.random.randn(10)
y_pred = y + .5 * np.random.randn(10)

print('MSE: %f' % metrics.mean_squared_error(y, y_pred))
print('MAE: %f' % metrics.mean_absolute_error(y, y_pred))
print('R^2: %f' % metrics.r2_score(y, y_pred))

In [ ]:
# y'yi yeniden ölçeklendir

y = 2 * y
y_pred = 2 * y_pred

print('MSE: %f' % metrics.mean_squared_error(y, y_pred))
print('MAE: %f' % metrics.mean_absolute_error(y, y_pred))
print('R^2: %f' % metrics.r2_score(y, y_pred))

# **Sınıflandırma Görevleri için Metrikler**

Regresyon için metrikler bir hata teriminin hesaplanmasına dayanır (yani tahminimiz ile temel gerçek arasındaki fark). Bunu bir sınıflandırma görevi için yapamayız, bu nedenle sınıflandırma için tamamen farklı metrikler tanımlamamız gerekecektir. Bir tahminde bulunduğumuzda olası sonuçlarla başlayalım.

Metriğimizi oluşturmak için kullanabileceğimiz dört olası sonucumuz var. Biz sadece üç olasılığı ele alacağız (daha birçok metrik tanımlanmış olsa da).

**Accuracy** en sezgisel olanıdır: doğru pozitiflerin ve negatiflerin oranının miktarıdır. Doğru pozitifleri ve doğru negatifleri toplar ve toplam tahmin sayısına böleriz.

**Accuracy**, sınıf dengesizliğinin olduğu görevlerden zarar görür. Örneğin, dolandırıcılık tespitinde gerçek pozitifler çok nadirdir. Bu nedenle, sadece her zaman negatif tahmin ederek yüksek doğruluk elde edebiliriz. Tüm gözlemlerin yalnızca %0,1'i gerçekten pozitifse, her zaman negatifi tahmin eden bir model %99,9 doğruluk elde eder, ancak bu açıkça dolandırıcılığı tespit etmek için kötü bir modeldir.

Bu örnek, genellikle bir sınıfı diğerinden daha fazla önemsediğimizi göstermektedir. Örneğin, bir işlemin hileli olduğunu düşünüyorsak, bunu araştırmak için bazı kaynakları boşa harcayabiliriz, ancak bir dolandırıcılık vakasını kaçırmak çok daha pahalıya mal olabilir. Bu durumda en çok yanlış negatiflerden kaçınmak isteyebiliriz.

**Recall**, gerçek pozitiflerin sayısının gerçek pozitiflerin sayısına bölünmesiyle elde edilir. Yanlış negatiflerin sayısı düşük olduğu sürece, çok fazla gerçek pozitif olmasa bile geri çağırma 1'e yakın olacaktır.

Öte yandan, bir dolandırıcılık davası mahkemeye giderse, bir sanığı haksız yere cezalandırmak istemeyiz. Bu durumda yanlış pozitiflerden kaçınmak önemlidir. **Precision**, doğru pozitiflerin sayısının pozitif tahminlerin sayısına bölünmesiyle elde edilir. Yanlış pozitiflerin sayısı düşük olduğu sürece, hassasiyet 1'e yakın olacaktır.



In [ ]:
y = [0, 0, 1, 0, 1, 1, 0, 1]
y_pred = [0, 1, 1, 0, 1, 1, 0, 1]

print('Accuracy: %f' % metrics.accuracy_score(y, y_pred))
print('Recall: %f' % metrics.recall_score(y, y_pred))
print('Precision: %f' % metrics.precision_score(y, y_pred))

# **Precision - Recall**

Modelimizi ayarlarken, pozitif tahminleri negatif tahminlerle değiştirerek **precision** ve **recall** arasında bir değiş tokuş söz konusudur.

Genellikle sınıflandırma modelimiz bir gözlemin bir sınıfta mı yoksa diğerinde mi olduğunu tahmin etmez, bunun yerine gözlemin bir sınıfta veya diğerinde olma olasılığını tahmin eder. Gözlemin pozitif sınıfta olduğunu tahmin edeceğimiz ve altında negatif olduğunu tahmin edeceğimiz bir eşik olasılığı(**threshold**) seçeriz.

In [ ]:
p_pred = np.linspace(0, 1, 1000)
y = np.random.binomial(1, p_pred)


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from ipywidgets import interact

def plot_threshold(threshold=0.5):
    true_pos = (p_pred > threshold) & (y == 1)
    true_neg = (p_pred < threshold) & (y == 0)
    false_pos = (p_pred > threshold) & (y == 0)
    false_neg = (p_pred < threshold) & (y == 1)

    plt.plot(p_pred[true_pos], y[true_pos], '.', label='True positive')
    plt.plot(p_pred[false_pos], y[false_pos], '.', label='False positive')
    plt.plot(p_pred[true_neg], y[true_neg], '.', label='True negative')
    plt.plot(p_pred[false_neg], y[false_neg], '.', label='False negative')

    plt.axvline(threshold)
    plt.ylim(-0.25, 1.25)
    plt.legend()

    precision = 1
    if (p_pred > threshold).sum() > 0:
        precision = float(true_pos.sum()) / (true_pos.sum() + false_pos.sum())
    recall = float(true_pos.sum()) / (true_pos.sum() + false_neg.sum())

    plt.title('Precision: %0.2f | Recall: %0.2f' % (precision, recall))

interact(plot_threshold, threshold=(0, 1, 0.1));
     

In [ ]:
precisions, recalls, thresholds = metrics.precision_recall_curve(y, p_pred)

plt.plot(recalls, precisions)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision v. Recall');

Bu eğriyi tek bir sayı ile özetleyebiliriz: eğrinin altındaki alan. Modelimiz mükemmel olsaydı, eşikten bağımsız olarak hassasiyet ve geri çağırma değerlerinin her ikisi de 1 olurdu, dolayısıyla eğrinin altındaki alan 1 olurdu. Modelimiz her zaman yanlış olsaydı, eşikten bağımsız olarak hassasiyet ve geri çağırma 0 olurdu, dolayısıyla eğrinin altındaki alan 0 olurdu. Eşikten bağımsız olarak modelimiz ne kadar iyi olursa, eğrinin altındaki alan 1'e o kadar yakın olur. Sonunda bir eşik seçmemiz gerekir ve hassasiyet veya geri çağırmaya öncelik vermeyi seçebiliriz, ancak hassasiyet-geri çağırma eğrisinin altındaki alan (AUC), sınıflandırma görevlerinde model performansını değerlendirmek için çok yararlı bir metriktir.
